In [0]:
# IMPORTS
import imageio
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

from skimage import io, img_as_float, img_as_uint
from skimage.color import rgb2gray
from skimage.transform import resize, rescale
from matplotlib import pyplot as plt
from google.colab import files

In [0]:
# VIDEO PARAMS
horizontal_slice = 285
output_size = 256
offset_frames = 10000

In [0]:
!rm -r F1_test_256_rgb
!rm -r F1_train_256_rgb
!rm *.zip
!rm *.tfrecord

# COMMON METHODS

In [0]:
# FUNCTION TO GENERATE ALL THE RGB FRAMES FROM A VIDEO
def generate_all_frames(video):
    count = -1
    total_frames = video.get_length()
    original_shape = video.get_data(0).shape
    scale = output_size / horizontal_slice
    
    for img in video:
        count += 1
        if count >= total_frames: break
        
        f = img_as_float(img[:horizontal_slice, :])
        f = resize(f,(horizontal_slice, horizontal_slice, 3), mode='reflect')
        f = rescale(f, scale=scale, mode='reflect')
        f = f - f.min()
        yield f/f.max()

In [0]:
# FUNCTION TO GENERATE ALL THE EXAMPLES (image, label) FOR EACH VIDEO
def generate_all_examples(videos):
    offset = 0
    for video_file in videos:
        video = imageio.get_reader(video_file, 'ffmpeg', mode='I')
        for n,f in enumerate(generate_all_frames(video)):
            yield f, n+offset
        offset += offset_frames
        print('Done generating all the frames of', video_file)
        print('Next offset',offset)

# TRAIN IMAGES

In [0]:
%%time
# UPLOAD ALL TRAINING VIDEOS
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} KB'.format(
        name=fn, length=len(uploaded[fn])/1000 ))

In [0]:
train_videos = []
for vid_name in uploaded.keys():
    basePath = os.getcwd()
    filename = basePath + '/' + vid_name
    train_videos.append(filename)
print(train_videos)

In [0]:
%%time
# LOAD, PROCESS AND SAVE ALL THE TRAINING IMAGES
train_dir = basePath + '/F1_train_'+str(output_size)+'_rgb'

# MAKE DIR FOR THE TRAINING IMAGES IF IT DOES NOT EXIST
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    
# COMPUTE AND SAVE
for img, label in generate_all_examples(train_videos):
    filename = 'frame_'+str(label)+'.png'
    io.imsave(train_dir+'/'+filename, img_as_uint(img))

In [0]:
# ZIP THE TRAIN FOLDER AND DOWNLOAD (OPTIONAL)
!zip -r $train_dir'.zip' $train_dir

# TEST IMAGES

In [0]:
%%time
# UPLOAD ALL TEST VIDEOS
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} KB'.format(
        name=fn, length=len(uploaded[fn])/1000 ))

In [0]:
test_videos = []
for vid_name in uploaded.keys():
    basePath = os.getcwd()
    filename = basePath + '/' + vid_name
    test_videos.append(filename)
print(test_videos)

In [0]:
%%time
# LOAD, PROCESS AND SAVE ALL THE TEST IMAGES
test_dir = basePath + '/F1_test_'+str(output_size)+'_rgb'

# MAKE DIR FOR THE TEST IMAGES IF IT DOES NOT EXIST
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
# COMPUTE AND SAVE
for img, label in generate_all_examples(test_videos):
    filename = 'frame_'+str(label)+'.png'
    io.imsave(test_dir+'/'+filename, img_as_uint(img))

In [0]:
# ZIP THE TEST FOLDER AND DOWNLOAD (OPTIONAL)
!zip -r $test_dir'.zip' $test_dir